In [ ]:
# Downloads necessários
!pip install transformers
!pip install einops accelerate bitsandbytes
!pip install sentence_transformers
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install colabcode
!pip install fastapi

# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from google.colab import userdata
import os
from colabcode import ColabCode
from fastapi import FastAPI
import logging



In [4]:
# Definindo a variável de ambiente e tokens

os.system(f"huggingface-cli login --token {userdata.get('HF_TOKEN')}")

os.environ["NGROK_AUTHTOKEN"] = userdata.get('NGROK_AUTHTOKEN')

0

## Processamento de PDF e Criação de Vetores de Texto


In [7]:
# Carregando o PDF
loader = PyPDFLoader('/content/protocolo-clinico-e-diretrizes-terapeuticas-do-tabagismo.pdf')
pages = loader.load()

# Dividindo o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(pages)

# Configurações do modelo de embeddings
model_name = "tgsc/sentence-transformer-ult5-pt-small"
model_kwargs = {"device": "cuda"}

# Inicializando os embeddings
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Criando o banco de vetores
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.80k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/490k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.19M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Modelo llm bode

In [4]:
# Define the language model to use
llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br-no-peft'

# Load the pre-trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained(llm_model, trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(llm_model)

# Set the model to evaluation mode
model.eval()

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
 

In [9]:
# Criando o pipeline para geração de texto
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # O langchain espera o texto completo
    task='text-generation',
    # Parâmetros do modelo
    # stopping_criteria=stopping_criteria,  # Sem isso, o modelo pode gerar saídas repetitivas
    max_new_tokens=1024,  # Número máximo de tokens a serem gerados na saída
    repetition_penalty=1.1,  # Evita repetições excessivas na saída
    temperature=0.1,
    top_p=0.75,
    num_beams=2,
    # do_sample=True,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Criando um pipeline específico para o seu modelo
llm = HuggingFacePipeline(pipeline=generate_text)


## Definir Prompt e RAG com ChromaDB

In [11]:
# Prompt personalizado
template = """
Use o contexto a seguir (delimitado por <ctx></ctx>) e o histórico do chat (delimitado por <hs></hs>) para responder à pergunta:
------
<ctx> {context} </ctx>
------
<hs> {history} </hs>
------
{question}
Resposta:

"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# Comentário: O código acima define um template personalizado para gerar respostas com base em contexto e histórico do chat. Ele utiliza variáveis como "history", "context" e "question" para preencher os campos relevantes no template.


In [12]:
# Configurando o RetrievalQA com os parâmetros desejados
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',  # Substitua pelo tipo de cadeia desejado
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,  # Substitua pelo prompt correto
        "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"
        ),
    }
)

# Definindo a pergunta
pergunta = 'O que fazer no caso de recaída?'

# Invocando o RetrievalQA para obter a resposta
result = qa.invoke(pergunta)

In [28]:
print(result['result'])


Use o contexto a seguir (delimitado por <ctx></ctx>) e o histórico do chat (delimitado por <hs></hs>) para responder à pergunta:
------
<ctx> O tratamento do tabagismo pode ser iniciado em qualquer nível de ate nção à 
saúde, sendo preferencialmente na Atenção Primária à Saúde (APS). Ao término do 
tratamento , caso seja alcançada a abstinência, recomenda -se que o paciente seja 
acompanhado por até um ano para o apoio na manutenção deste quadro.  
No caso de recaída, deve -se reavaliar o tratamento feito anteriormente e 
decidir  em conjunto  como  proceder  em relação  ao retratamento.  Sugere -se ainda  a busca  
ativa  de pacientes que tenham abandonado o  tratamento.  
Os possíveis efeitos adversos pelo uso dos  medicamentos, já relatados, 
devem ser acompanhados pelo profissional da saúde responsável pelo atendimento, que irá 
decidir sobre  a melhor conduta a ser adotada em cada caso, fazendo , quando necessário, 
encaminhamentos para outros profissionais ou unidades de saúde. 

## FastAPI com URL Pública

In [21]:
app = FastAPI(title="Llm Bode as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# Initialize logging
my_logger = logging.getLogger()
my_logger.setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = qa

@app.post("/api", tags=["prompt"])
async def get_prompt(pergunta):
    try:
        resultado = model.invoke(pergunta)
        print(resultado)
        return resultado
    except Exception as e:
        my_logger.error(f"Something went wrong: {str(e)}")
        return {"resultado": "error"}

<ipython-input-21-4631d12a21ab>:13: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
app = FastAPI(title="Llm Bode as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# Inicialização do modelo
model = None

@app.on_event("startup")
async def load_model():
    global model
    model = qa

@app.on_event("shutdown")
async def close_model():
    # Lógica para liberar recursos ou salvar dados antes de encerrar o aplicativo
    pass

@app.post("/api", tags=["prompt"])
async def get_prompt(pergunta):
    try:
        resultado = model.invoke(pergunta)
        print(resultado)
        return resultado
    except Exception as e:
        # Lidar com exceções e registrar erros
        my_logger.error(f"Something went wrong: {str(e)}")
        return {"resultado": "erro"}



In [22]:
cc = ColabCode(port=12000, code=False)
cc.run_app(app=app)

DEBUG:pyngrok.process:Removing stale process for "ngrok_path" /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok
DEBUG:pyngrok.process:ngrok process starting with PID: 7425
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="client session established" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:25+0000 lvl=info msg="tunnel session started" obj=tunnels.session
DEBUG:pyngrok.process:ngrok process has started with API URL: http://127.0.0.1:404

Public URL: NgrokTunnel: "https://7718-35-199-181-49.ngrok-free.app" -> "http://localhost:12000"


INFO:     Started server process [1013]
INFO:uvicorn.error:Started server process [1013]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     2804:14c:ba:af0e:25e1:5a07:e9f5:5d3a:0 - "GET /docs HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:37+0000 lvl=info msg="join connections" obj=join id=70aeb30873ba l=127.0.0.1:12000 r=[2804:14c:ba:af0e:25e1:5a07:e9f5:5d3a]:63943


INFO:     2804:14c:ba:af0e:25e1:5a07:e9f5:5d3a:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-04-07T20:42:48+0000 lvl=info msg="join connections" obj=join id=f5752e072365 l=127.0.0.1:12000 r=[2804:14c:ba:af0e:25e1:5a07:e9f5:5d3a]:63951




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use o contexto a seguir (delimitado por <ctx></ctx>) e o histórico do chat (delimitado por <hs></hs>) para responder à pergunta:
------
<ctx> a expectativa de vida, melhorar a saúde e a qualidade de vida. Aqueles idosos que decidem 
fazer uma tentativa têm elevada taxa de sucesso (35,36). 
 
6.5. Pacientes com tuberculose  
Segundo dados da OMS, mais de 20% da incidência global de tuberculose 
pode ser atribuída ao tabagismo, que aumenta em duas vezes e meia o risco d essa doença. 
Tabagistas sem história anterior de tuberculose apresentam risco de morte por essa  doença

ser uma parte integrante do cuidado do paciente com tuberculose ( 38-40). 
É indicado  que portadores  de tuberculose,  em qualquer  fase do tratamento,  
sejam  tratados  da dependência à nicotina e utilizem as mesmas medidas  terapêuticas 
empregadas para a população em geral. O tratamento preconizado é a assoc

INFO:pyngrok.process.ngrok:t=2024-04-07T20:43:52+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:pyngrok.process.ngrok:t=2024-04-07T20:43:52+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [1013]
INFO:uvicorn.error:Finished server process [1013]
